## Generating a mock bank database, and deploying it on the cloud (Amazon RDS)

### Gil Harari - [LinkedIn](https://www.linkedin.com/in/gilharari/); [GitHub](https://github.com/gilha); [Mail](mailto:gil1996@gmail.com); Phone: +972-52-744-1568


In [42]:
import pandas as pd
import numpy as np
import datetime
from faker import Faker
from sqlalchemy import create_engine
import psycopg2
from psycopg2 import sql

fake = Faker()

Just for fun, instead of purely random data, let's generate our samples as if they're normally distributed:

In [35]:
def get_normal_dist_sample(low: float, high: float, return_int=False) -> float:
    mean = (high - low)/2
    std = (high - low)/4 # "The Range Rule of Thumb"
    while True:
        sample = np.round(np.random.normal(loc=mean, scale=std), 3)
        if return_int:
            sample = int(sample)
        if low <= sample <= high:
            return sample

In [ ]:
# clients params:
EDUCATION_LEVELS = ['Elementary School', 'High School', "Bachelor's Degree", "Master's Degree", 'Doctorate']
TOTAL_CLIENTS = 50
JOIN_DATE_LIMIT = '-1y'

## 1. Generating the tables:

In [30]:
clients = {}
for costumer_id in range(TOTAL_CLIENTS):
    client_data = {}
    client_data['join_date'] = fake.date_between(start_date=JOIN_DATE_LIMIT, end_date='today')
    client_data['age'] = get_normal_dist_sample(low=15, high=85, return_int=True)
    client_data['gender'] = np.random.choice(['M', 'F'])
    client_data['income'] = get_normal_dist_sample(low=10000, high=1000000)
    client_data['credit_score'] = get_normal_dist_sample(low=300, high=850, return_int=True)
    client_data['education'] = np.random.choice(EDUCATION_LEVELS)
    clients[costumer_id] = client_data

df_clients = pd.DataFrame.from_dict(clients, orient='index')
df_clients

,join_date,age,gender,income,credit_score,education
0,2022-10-11,36,M,945339.126,369,Master's Degree
1,2022-05-17,26,F,832783.424,319,Bachelor's Degree
2,2022-07-05,73,F,190972.967,300,Doctorate
3,2023-02-21,51,F,461201.942,416,Master's Degree
4,2023-01-02,19,F,752608.159,322,Doctorate
5,2023-02-24,25,F,647081.774,360,Elementary School
6,2022-11-07,37,M,492702.438,394,Doctorate
7,2022-04-23,16,M,382060.660,313,Doctorate
8,2023-01-14,36,M,702775.207,421,Doctorate
9,2022-10-30,34,F,412912.786,324,Master's Degree


In [31]:
financials = {}

financials_id = 0
for costumer_id in df_clients.index:
    days_count = (datetime.date.today() - df_clients['join_date'][costumer_id]).days
    while days_count >= 0:
        client_financials_data = {}
        client_financials_data['customer_id'] = costumer_id
        client_financials_data['date_id'] = (datetime.datetime.today() - datetime.timedelta(days=days_count)).date()
        client_financials_data['balance_amount'] = get_normal_dist_sample(low=-10000, high=10000000)
        client_financials_data['deposits_amount'] = get_normal_dist_sample(low=0, high=100000)
        client_financials_data['withdrawal_amount'] = get_normal_dist_sample(low=0, high=100000)
        client_financials_data['credit_card_spending'] = get_normal_dist_sample(low=0, high=100000)
        client_financials_data['debit_card_spending'] = get_normal_dist_sample(low=0, high=100000)
        client_financials_data['cash_spending'] = get_normal_dist_sample(low=0, high=100000)
        financials[financials_id] = client_financials_data
        days_count -= 1
        financials_id += 1

df_financials = pd.DataFrame.from_dict(financials, orient='index')
df_financials

,customer_id,date_id,balance_amount,deposits_amount,withdrawal_amount,credit_card_spending,debit_card_spending,cash_spending
0,0,2022-10-11,3770080.871,12590.179,44769.336,67056.918,75333.319,26410.254
1,0,2022-10-12,1997732.614,68899.403,53377.161,75286.466,12602.904,14318.761
2,0,2022-10-13,5647910.544,58860.090,68579.243,67464.790,35215.101,38322.044
3,0,2022-10-14,5129966.914,23146.673,97.023,45199.385,7976.211,33164.143
4,0,2022-10-15,4893753.949,49595.881,91624.785,40275.613,18323.838,36260.721
...,...,...,...,...,...,...,...,...
8685,49,2023-03-13,3198756.169,33945.195,49050.008,98876.543,23143.436,78607.433
8686,49,2023-03-14,3964040.396,59850.960,62431.426,68375.718,30278.012,58478.618
8687,49,2023-03-15,3142808.805,53948.648,63712.548,80708.615,32844.559,42551.943
8688,49,2023-03-16,8004697.776,27940.579,58731.744,69639.404,37808.316,61488.191


In [33]:
ledger = {}
ledger_id = 0
for costumer_id in df_clients.index:
    amounts_of_client_loans = np.random.randint(low=-3, high=12)
    if amounts_of_client_loans < 0:
        amounts_of_client_loans = 0 # so we'll have some more clients who didn't took any loans
    for loan in range(amounts_of_client_loans):
        loan_data = {}
        loan_data['customer_id'] = costumer_id
        loan_data['loan_size'] = get_normal_dist_sample(low=10000, high=10000000, return_int=True)
        loan_data['loan_start_date'] = fake.date_between(start_date=df_clients['join_date'][costumer_id], end_date='today')
        ledger[ledger_id] = loan_data
        ledger_id += 1

df_ledger = pd.DataFrame.from_dict(ledger, orient='index')
df_ledger

,customer_id,loan_size,loan_start_date
0,0,4459330,2022-12-15
1,0,7158201,2023-03-13
2,0,4996256,2022-10-30
3,0,1987425,2022-10-29
4,0,4683786,2023-02-08
...,...,...,...
154,49,3088000,2022-11-13
155,49,5190533,2023-02-02
156,49,6819395,2022-10-17
157,49,6484935,2022-11-01


## 2. Deploying the data to Amazon RDS:

In [61]:
# Params:
PGENDPOINT = 'insaitdb.cccnbalmnxxd.eu-north-1.rds.amazonaws.com' # aka host
PGDATABASE_NAME = 'insaitdb'
PGUSER_NAME = 'postgres'
PGPASSWORD = 'postgres'

def connect():
    conn_string = "host="+ PGENDPOINT +" port="+ "5432" +" dbname="+ PGDATABASE_NAME +" user=" + PGUSER_NAME \
                  +" password="+ PGPASSWORD
    
    conn = psycopg2.connect(conn_string)
    print("Connected!")

    cursor = conn.cursor()
    
    return conn, cursor

In [57]:
conn, cursor = connect()

Connected!


In [44]:
query_create_tables = sql.SQL("""CREATE TABLE clients (
  customer_id INT PRIMARY KEY,
  join_date DATE,
  age INT,
  gender VARCHAR(10),
  income DECIMAL(20,3),
  credit_score INT,
  education VARCHAR(280)
);

CREATE TABLE financials (
  customer_id INT,
  date_id DATE,
  balance_amount DECIMAL(20,3),
  deposits_amount DECIMAL(20,3),
  withdrawal_amount DECIMAL(20,3),
  credit_card_spending DECIMAL(20,3),
  debit_card_spending DECIMAL(20,3),
  cash_spending DECIMAL(20,3),
  FOREIGN KEY (customer_id) REFERENCES Clients(customer_id)
);

CREATE TABLE ledger (
  customer_id INT,
  loan_size DECIMAL(20,3),
  loan_start_date DATE,
  FOREIGN KEY (customer_id) REFERENCES Clients(customer_id)
);""")

cur = conn.cursor()
cur.execute(query_create_tables)
cur.close()
conn.commit()

In [59]:
def insert_df_to_table(conn, df, table):
    tpls = [tuple(x) for x in df.to_numpy()]
    
    cols = ','.join(list(df.columns))
    
    if table == 'clients':
        sql = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s,%%s,%%s, %%s, %%s)" % (table, cols)
    elif table == 'financials':
        sql = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s,%%s,%%s, %%s, %%s, %%s)" % (table, cols)
    elif table == 'ledger':
        sql = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s)" % (table, cols)
    cursor = conn.cursor()
    cursor.executemany(sql, tpls)
    print("Successfully inserted data to " + table)
    cursor.close()
    conn.commit()

insert_df_to_table(conn, df_clients.reset_index().rename(columns={"index": "customer_id"}), 'clients')
insert_df_to_table(conn, df_financials, 'financials')
insert_df_to_table(conn, df_ledger, 'ledger')

Successfully inserted data to clients
Successfully inserted data to financials
Successfully inserted data to ledger


### 3. Creating a Ledger <> Financials view:

In [60]:
query_create_view = """CREATE VIEW financial_status_on_loan_start AS
SELECT f.customer_id,
    f.date_id,
    f.balance_amount,
    f.deposits_amount,
    f.withdrawal_amount,
    f.credit_card_spending,
    f.debit_card_spending,
    f.cash_spending,
    l.loan_size
FROM financials f
JOIN ledger l ON f.customer_id = l.customer_id
AND f.date_id = l.loan_start_date;
"""
cur = conn.cursor()
cur.execute(query_create_view)
cur.close()
conn.commit()